In [1]:
import psycopg2
import pandas as pd
import sqlalchemy

In [18]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="Adventureworks", 
                            user="postgres", password="postgres")
cur = conn.cursor()

In [19]:


cur.execute('''
           create table if not exists  customer 
            (customer_id int,
             store_id int,
             first_name varchar,
             last_name varchar)
            ''')


cur.execute('''
           insert into   customer
            values(1,1,'Mike','Mike');
            ''')


cur.execute('''
           select* from customer
        
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike
1,1,1,Mike,Mike


In [20]:
cur.execute('''
           CREATE TEMP TABLE temp_customer AS
        SELECT customer_id
             , store_id
             , first_name
             , last_name
        FROM   public.customer
        WHERE  store_id =1;
            ''')

In [21]:
query = f'''
        select * from temp_customer;
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike
1,1,1,Mike,Mike


### использование временной таблицы в функции

In [22]:
cur.execute('''
            Create or replace function use_temp_table() returns 
            table(first_name text, last_name text) as
            $$
            begin
            DROP TABLE IF EXISTS temp_table CASCADE;
            CREATE TEMP TABLE temp_table  AS
            SELECT customer.first_name
                 , customer.last_name
            FROM   customer
            WHERE  store_id =1;

             return query
                SELECT  temp_table.first_name::text, temp_table.last_name::text FROM 
                  temp_table;
            END
            $$ language plpgsql;
            ''')

cur.execute('''
            select * from  use_temp_table();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['first_name', 'last_name'])

,first_name,last_name
0,Mike,Mike
1,Mike,Mike


In [16]:
conn.rollback()
conn.close()

### Просмотр pg_locks показывает, какие блокировки предоставлены и какие процессы ожидают получения блокировок.


In [23]:
query = f'''
         select relation::regclass, * from pg_locks where not granted;
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,relation,locktype,database,relation,page,tuple,virtualxid,transactionid,classid,objid,objsubid,virtualtransaction,pid,mode,granted,fastpath,waitstart


### Следующий запрос может быть полезен, чтобы увидеть, какие процессы блокируют операторы SQL 
### они находят блокировки только на уровне строк, но не на уровне объектов).

In [24]:
query = f'''
          SELECT blocked_locks.pid     AS blocked_pid,
         blocked_activity.usename  AS blocked_user,
         blocking_locks.pid     AS blocking_pid,
         blocking_activity.usename AS blocking_user,
         blocked_activity.query    AS blocked_statement,
         blocking_activity.query   AS current_statement_in_blocking_process
   FROM  pg_catalog.pg_locks         blocked_locks
    JOIN pg_catalog.pg_stat_activity blocked_activity  ON blocked_activity.pid = blocked_locks.pid
    JOIN pg_catalog.pg_locks         blocking_locks 
        ON blocking_locks.locktype = blocked_locks.locktype
        AND blocking_locks.database IS NOT DISTINCT FROM blocked_locks.database
        AND blocking_locks.relation IS NOT DISTINCT FROM blocked_locks.relation
        AND blocking_locks.page IS NOT DISTINCT FROM blocked_locks.page
        AND blocking_locks.tuple IS NOT DISTINCT FROM blocked_locks.tuple
        AND blocking_locks.virtualxid IS NOT DISTINCT FROM blocked_locks.virtualxid
        AND blocking_locks.transactionid IS NOT DISTINCT FROM blocked_locks.transactionid
        AND blocking_locks.classid IS NOT DISTINCT FROM blocked_locks.classid
        AND blocking_locks.objid IS NOT DISTINCT FROM blocked_locks.objid
        AND blocking_locks.objsubid IS NOT DISTINCT FROM blocked_locks.objsubid
        AND blocking_locks.pid != blocked_locks.pid
    JOIN pg_catalog.pg_stat_activity blocking_activity ON blocking_activity.pid = blocking_locks.pid
   WHERE NOT blocked_locks.granted;
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,blocked_pid,blocked_user,blocking_pid,blocking_user,blocked_statement,current_statement_in_blocking_process


### Эффекты команды SET LOCAL для переменной ограничиваются процедурой\функцией, внутри которой выполняется команда; 
### предыдущее значение параметра конфигурации восстанавливается после выхода из процедуры. 
###  Эффекты конфигурации сохранятся после выхода из процедуры\функции, 
### если текущая транзакция не будет отменена
### console

In [26]:
cur.execute('''
            CREATE OR REPLACE PROCEDURE datestyle_change() 
            LANGUAGE plpgsql 
            SET datestyle TO postgres, dmy
            AS $$
                BEGIN
                    RAISE NOTICE 'Current Date is : % ', now();
                END;
            $$ ;

            ''')

cur.execute('''
            call datestyle_change();
            ''')

print(conn.notices)

['NOTICE:  relation "customer" already exists, skipping\n', 'NOTICE:  table "temp_table" does not exist, skipping\n', 'NOTICE:  Current Date is : Sat 22 Jul 19:26:39.891879 2023 UTC \n']
